<a href="https://colab.research.google.com/github/laurieye/Prescriptive-Analytics/blob/main/Project_Applied_Electronics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
# Given Data
costs = np.array([
    [92.63, 104.03, 145.95, 112.43, 107.80, 112.19],
    [160.20, 93.25, 148.88, 113.61, 103.45, 111.85],
    [186.70, 122.31, 112.31, 135.98, 127.76, 133.35],
    [127.34, 84.84, 122.51, 73.34, 87.84, 91.04],
    [152.64, 95.15, 144.73, 107.62, 89.15, 107.00],
    [252.78, 162.24, 236.36, 177.62, 168.96, 149.24]
])

capacity = np.array([22.0, 3.7, 4.5, 47.0, 18.5, 5.0])
demand = np.array([3.00, 2.60, 16.00, 20.00, 26.40, 11.90])


# VAM function
def vam(costs, capacity, demand):
    allocation = np.zeros((len(capacity), len(demand))) #Initialize an allocation matrix for output
    costs_copy = np.array(costs)

    while sum(capacity) > 0 and sum(demand) > 0:
        # Calculate the opportunity costs (difference between min and 2nd min) for each row and column
        row_diff = []
        col_diff = []

        # Compute the opportunity costs for rows
        for row in costs_copy:
            sorted_row = sorted([x for x in row if x != float('inf')])
            if len(sorted_row) >= 2:
                row_diff.append(sorted_row[1] - sorted_row[0])
            else:
                row_diff.append(0)

        # Compute the opportunity costs for columns
        for i in range(len(demand)):
            sorted_col = sorted([x for x in costs_copy[:, i] if x != float('inf')])
            if len(sorted_col) >= 2:
                col_diff.append(sorted_col[1] - sorted_col[0])
            else:
                col_diff.append(0)

        # Find the maximum opportunity cost
        if max(row_diff) > max(col_diff):
          idx = row_diff.index(max(row_diff))
          j = np.argmin(costs_copy[idx, :])
        else:
          max_col_diff = max(col_diff)
          j = col_diff.index(max_col_diff)

        # Deal with ties - if a tie exists, break the tie by picking the first occurence
        for i in range(len(col_diff)):
          if col_diff[i] == max_col_diff:
            j = i
            break
        idx = np.argmin(costs_copy[:, j])

        # Assign the minimum of demand and capacity to the cell
        alloc = min(capacity[idx], demand[j])
        allocation[idx][j] = alloc
        capacity[idx] -= alloc
        demand[j] -= alloc

        # Set the costs of exhausted row or column to a high value so it is no longer used
        if capacity[idx] == 0:
            costs_copy[idx, :] = float('inf')
        if demand[j] == 0:
            costs_copy[:, j] = float('inf')

    return allocation


# Q3: 100% capacity
result_100 = vam(costs, capacity.copy(), demand.copy())
print("Framework Matrix for 100% capacity:")
print(result_100)
cost_result_100 = result_100 * costs / 100
print(f"\nThe projected cost for 100% capacity: \n{cost_result_100}")
print(f"The projected cost for 100% capacity: {round(np.sum(cost_result_100),3)} million dollars")


# Q4: 85% capacity
capacity_85 = [s * 0.85 for s in capacity]
result_85 = vam(costs, capacity_85.copy(), demand.copy())
print("\nFramework Matrix for 85% capacity:")
print(result_85)
cost_result_85 = result_85 * costs / 100
print(f"\nThe projected cost for 85% capacity: \n{cost_result_85}")
print(f"The projected cost for 85% capacity: {round(np.sum(cost_result_85),3)} million dollars")


# Q5: 90% capacity, with Chile
capacity_90 = [s * 0.9 for s in capacity]

result_with_chile_90 = vam(costs, capacity_90.copy(), demand.copy())
print("Framework Matrix with Chile and at 90% capacity:")
print(result_with_chile_90)
cost_Chile_90 = result_with_chile_90 * costs
print(f"\nThe projected cost for 90% capacity with Chile: \n{cost_Chile_90}")
print(f"The projected cost for 90% capacity with Chile: {round(np.sum(cost_Chile_90)/100,3)} million dollars")


# Q5: 90% capacity, without Chile
reduced_capacity = [22.0, 3.7, 0, 47.0, 18.5, 5.0] #Set capacity of Chile to 0
reduced_capacity_90 = [s * 0.9 for s in reduced_capacity]

result_without_chile = vam(costs, reduced_capacity_90.copy(), demand.copy())
print("Framework Matrix without Chile and at 90% capacity:")
print(result_without_chile)
cost_noChile_90 = result_without_chile * costs / 100
print(f"\nThe projected cost for 90% capacity without Chile: \n{cost_noChile_90}")
print(f"The projected cost for 90% capacity without Chile: {round(np.sum(cost_noChile_90),3)} million dollars")

Framework Matrix for 100% capacity:
[[ 3.   0.   0.   0.   3.2  0. ]
 [ 0.   0.   0.   0.   3.7  0. ]
 [ 0.   0.   4.5  0.   0.   0. ]
 [ 0.   2.6 11.5 20.   1.  11.9]
 [ 0.   0.   0.   0.  18.5  0. ]
 [ 0.   0.   0.   0.   0.   0. ]]

The projected cost for 100% capacity: 
[[ 2.7789   0.       0.       0.       3.4496   0.     ]
 [ 0.       0.       0.       0.       3.82765  0.     ]
 [ 0.       0.       5.05395  0.       0.       0.     ]
 [ 0.       2.20584 14.08865 14.668    0.8784  10.83376]
 [ 0.       0.       0.       0.      16.49275  0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]]
The projected cost for 100% capacity: 74.278 million dollars

Framework Matrix for 85% capacity:
[[ 3.     0.     4.125  0.    10.13   0.   ]
 [ 0.     2.6    0.     0.     0.545  0.   ]
 [ 0.     0.     3.825  0.     0.     0.   ]
 [ 0.     0.     8.05  20.     0.    11.9  ]
 [ 0.     0.     0.     0.    15.725  0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]]

The projected